# 📊 Preparação de Dados para Fine-Tuning

Este notebook prepara datasets para fine-tuning de LLMs.

**Execute LOCALMENTE** (não precisa GPU)

## O que este notebook faz:

1. ✅ Carrega dados de diferentes fontes (JSON, CSV, logs RAG)
2. ✅ Formata no template correto (Alpaca, ChatML, Llama2)
3. ✅ Cria splits treino/validação
4. ✅ Valida qualidade dos dados
5. ✅ Exporta para upload no Colab

## Fluxo:

```
Dados Brutos → Formatação → Validação → Export → Upload no Colab
```


## 🔧 Setup


In [ ]:
import sys
sys.path.append('../')

from src.finetuning.data_prep import DatasetPreparator, format_instruction_dataset, create_sample_dataset
from pathlib import Path
import json
import pandas as pd


## 📁 Configuração de Caminhos


In [ ]:
# Diretórios
DATA_DIR = Path("../data/finetuning")
DATA_DIR.mkdir(parents=True, exist_ok=True)

RAW_DATA_DIR = DATA_DIR / "raw"
RAW_DATA_DIR.mkdir(exist_ok=True)

PROCESSED_DATA_DIR = DATA_DIR / "processed"
PROCESSED_DATA_DIR.mkdir(exist_ok=True)

print(f"📁 Dados brutos: {RAW_DATA_DIR}")
print(f"📁 Dados processados: {PROCESSED_DATA_DIR}")
